In [1]:
import chess
import chess.svg
import random
import np
from time import sleep
from tkinter import *
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPDF, renderPM
from PIL import Image, ImageTk, PngImagePlugin

In [2]:
board_coord = [["A8","B8","C8","D8","E8","F8","G8","H8"],
               ["A7","B7","C7","D7","E7","F7","G7","H7"],
               ["A6","B6","C6","D6","E6","F6","G6","H6"],
               ["A5","B5","C5","D5","E5","F5","G5","H5"],
               ["A4","B4","C4","D4","E4","F4","G4","H4"],
               ["A3","B3","C3","D3","E3","F3","G3","H3"],
               ["A2","B2","C2","D2","E2","F2","G2","H2"],
               ["A1","B1","C1","D1","E1","F1","G1","H1"]]
my_board = [["R","N","B","Q","K","B","N","R"],
            ["P","P","P","P","P","P","P","P"],
            [".",".",".",".",".",".",".","."],
            [".",".",".",".",".",".",".","."],
            [".",".",".",".",".",".",".","."],
            [".",".",".",".",".",".",".","."],
            ["p","p","p","p","p","p","p","p"],
            ["r","n","b","q","k","b","n","r"]]

cur_board_img = ""

cur_piece = ""  # UCI coordinates
cp_x = 0
cp_y = 0
cp = ""         # piece value
move_to = ""    # UCI coordinates
mt_x = 0
mt_y = 0
mt = ""

die_img = ""
move_counter = 0
my_moves = 0

In [3]:
def get_svg(svg_file) :
	drawing = svg2rlg(svg_file)
	renderPM.drawToFile(drawing,"tmp.png",fmt="PNG")

In [4]:
def select_piece(event) :               # shows available attacks/moves, pawns only show attacks
	global cur_piece,move_to,cp_x,cp_y,cp
	x_coord = event.x
	y_coord = event.y
	x_coord -= 31
	y_coord -= 31
	x = x_coord / 92
	y = y_coord / 92
	cp_x = int(x)
	cp_y = int(y)
	cp = my_board[cp_y][cp_x]
	cur_piece = board_coord[cp_y][cp_x]
	squares = board.attacks(getattr(chess,cur_piece))
	draw_board(squares)

In [5]:
def move_piece(event) :
    global cur_piece,move_to,move_counter,my_moves,mt_x,mt_y,mt
    x_coord = event.x
    y_coord = event.y
    x_coord -= 31
    y_coord -= 31
    x = x_coord / 92
    y = y_coord / 92
    mt_x = int(x)
    mt_y = int(y)
    mt = my_board[mt_y][mt_x]
    move_to = board_coord[int(y)][int(x)]
    squares = ""
    move_uci = cur_piece + move_to
    my_move = chess.Move.from_uci(move_uci.lower())

    if (my_move in board.legal_moves or True) : 

        if (is_capture()) :
            change_board()
            board.push(move=my_move)
            draw_board(squares)
            window.update()
            print_board()
        else :
            board.push(chess.Move.null())
            draw_board(squares)
            window.update()
            print_board()
        my_moves += 1

        if (my_moves < 3) :
            board.push(chess.Move.null())
        else :
            for i in range (0,3) :
                AI()
                window.update()
                if (i < 2) :
                    board.push(chess.Move.null())
            my_moves = 0

    else :
        print("!!!!!!!!!!!!!!!!!\nILLEGAL MOVE\n!!!!!!!!!!!!!!!!!")

#        if (chess.Board.is_checkmate() or chess.Board.is_stalemate()) :		# not working, implementing later
#            img = Image.open("game_over.png")
#            cur_board_img = ImageTk.PhotoImage(img)
#            panel.config(image=cur_board_img)

In [6]:
def AI() :
    global cur_piece,move_to,move_counter,cp,cp_x,cp_y,mt,mt_x,mt_y
    if (move_counter < 8) :
        cur_piece = board_coord[1][move_counter]
        move_to = board_coord[2][move_counter]
        cp_x = 0 + move_counter
        cp_y = 1
        cp = my_board[cp_y][cp_x]
        mt_x = 0 + move_counter
        mt_y = 2
        mt = my_board[mt_y][mt_x]
        move_counter += 1
        squares = ""
        move_uci = cur_piece + move_to
        my_move = chess.Move.from_uci(move_uci.lower())
        if (is_capture()) :
            change_board()
            print_board()
            board.push(move=my_move)
            draw_board(squares)
        else :
            board.push(chess.Move.null())
    else : 
        board.push(chess.Move.null())

In [7]:
def is_capture() :
    global cp,mt
    print("cur_piece = ",cp)
    print("move_to = ",mt)
    print()
    if (str(mt) != '.') :

        die = roll_die()
        print("Die result = ",die,"\n")

        if (str(cp.lower()) == "p") :
            if (mt.lower() == "p") :
                if (die >= 4) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "r") :
                if (die >= 6) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "b") :
                if (die >= 5) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "n") :
                if (die >= 6) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "q") :
                if (die >= 6) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "k") :
                if (die >= 6) :
                    graveyard(mt)
                    return True
                else :
                    return False
            else :
                print("!!!!!!!!!!!!!!!!!\nERROR in is_capture()\n!!!!!!!!!!!!!!!!!")
                return False

        elif (cp.lower() == "r") :
            if (mt.lower() == "p") :
                if (die >= 5) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "r") :
                if (die >= 6) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "b") :
                if (die >= 5) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "n") :
                if (die >= 5) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "q") :
                if (die >= 4) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "k") :
                if (die >= 4) :
                    graveyard(mt)
                    return True
                else :
                    return False
            else :
                print("!!!!!!!!!!!!!!!!!\nERROR in is_capture()\n!!!!!!!!!!!!!!!!!")
                return False

        elif (cp.lower() == "b") :
            if (mt.lower() == "p") :
                if (die >= 3) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "r") :
                if (die >= 5) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "b") :
                if (die >= 4) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "n") :
                if (die >= 5) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "q") :
                if (die >= 5) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "k") :
                if (die >= 5) :
                    graveyard(mt)
                    return True
                else :
                    return False
            else :
                print("!!!!!!!!!!!!!!!!!\nERROR in is_capture()\n!!!!!!!!!!!!!!!!!")
                return False

        elif (cp.lower() == "n") :
            if (mt.lower() == "p") :
                if (die >= 2) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "r") :
                if (die >= 5) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "b") :
                if (die >= 4) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "n") :
                if (die >= 4) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "q") :
                if (die >= 6) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "k") :
                if (die >= 6) :
                    graveyard(mt)
                    return True
                else :
                    return False
            else :
                print("!!!!!!!!!!!!!!!!!\nERROR in is_capture()\n!!!!!!!!!!!!!!!!!")
                return False
                
        elif (cp.lower() == "q") :
            if (mt.lower() == "p") :
                if (die >= 2) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "r") :
                if (die >= 5) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "b") :
                if (die >= 4) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "n") :
                if (die >= 4) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "q") :
                if (die >= 4) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "k") :
                if (die >= 4) :
                    graveyard(mt)
                    return True
                else :
                    return False
            else :
                print("!!!!!!!!!!!!!!!!!\nERROR in is_capture()\n!!!!!!!!!!!!!!!!!")
                return False
                
        elif (cp.lower() == "k") :
            if (mt.lower() == "p") :
                if (die >= 0) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "r") :
                if (die >= 5) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "b") :
                if (die >= 4) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "n") :
                if (die >= 4) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "q") :
                if (die >= 4) :
                    graveyard(mt)
                    return True
                else :
                    return False
            elif (mt.lower() == "k") :
                if (die >= 4) :
                    graveyard(mt)
                    return True
                else :
                    return False
            else :
                print("!!!!!!!!!!!!!!!!!\nERROR in is_capture()\n!!!!!!!!!!!!!!!!!")
                return False

        else :
            print("!!!!!!!!!!!!!!!!!\nERROR in is_capture()\n!!!!!!!!!!!!!!!!!")
            return False
    else :
        return True

In [8]:
def graveyard(piece) :
    print(piece,"captured\n")
    if (piece.islower()) :
        piece += "w"
    else :
        piece += "b"

    pw = 0
    rw = 0
    nw = 0
    bw = 0
    qw = 0
    kw = 0

    pb = 0
    rb = 0
    nb = 0
    bb = 0
    qb = 0
    kb = 0

    print(piece,"\n")

In [9]:
def roll_die() :
    global die_number,die_img
    roll_list = []
    counter = 6
    rand_num = ""
    while (counter > 0) :
        rand_num = random.randint(1,6)
        roll_list.append(rand_num)
        counter -= 1
        img = Image.open("{}.png".format(rand_num))
        die_img = ImageTk.PhotoImage(img)
        die.config(image=die_img)
        die.update()
        sleep(.2)
    img = Image.open("{}.png".format(rand_num))
    die_img = ImageTk.PhotoImage(img)
    die.config(image=die_img)
    die.update()
    return roll_list[5]
    
#    if (roll_list[5] > 3) :
#        return true
#    else :
#        return false

In [10]:
def draw_board(squares) :
    f = open ("tmp.svg","w")
    f.write(chess.svg.board(board,squares=squares,size=800))
    f.close	
    get_svg("tmp.svg")
    img = Image.open("tmp.png")
    cur_board_img = ImageTk.PhotoImage(img)
    panel.config(image=cur_board_img)
    panel.image=cur_board_img

In [11]:
def change_board() :
    global my_board,cp,cp_y,cp_x,mt_y,mt_x
    my_board[cp_y][cp_x] = "."
    my_board[mt_y][mt_x] = cp

In [12]:
def print_board() :
    global my_board
    for y in range(8) :
        for x in range(8) :
            print(my_board[y][x]," ",end="")
        print()
    print("\n----------------------------------------\n")

In [13]:
window = Tk()
window.title("AI Chess Variant")
window.geometry("1200x875")

board = chess.Board(chess.STARTING_BOARD_FEN)

die_number = StringVar()
die_number.set("")

pb1 = Label(window,image="",bg="grey").grid(row=0,column=1,padx=45)
pb2 = Label(window,image="",bg="grey").grid(row=1,column=1,padx=45)
pb3 = Label(window,image="",bg="grey").grid(row=2,column=1,padx=45)
pb4 = Label(window,image="",bg="grey").grid(row=3,column=1,padx=45)
pb5 = Label(window,image="",bg="grey").grid(row=4,column=1,padx=45)
pb6 = Label(window,image="",bg="grey").grid(row=5,column=1,padx=45)
pb7 = Label(window,image="",bg="grey").grid(row=6,column=1,padx=45)
pb8 = Label(window,image="",bg="grey").grid(row=7,column=1,padx=45)
rb1 = Label(window,image="",bg="grey").grid(row=0,column=0,padx=45)
nb1 = Label(window,image="",bg="grey").grid(row=1,column=0,padx=45)
bb1 = Label(window,image="",bg="grey").grid(row=2,column=0,padx=45)
qb  = Label(window,image="",bg="grey").grid(row=3,column=0,padx=45)
kb  = Label(window,image="",bg="grey").grid(row=4,column=0,padx=45)
bb2 = Label(window,image="",bg="grey").grid(row=5,column=0,padx=45)
nb2 = Label(window,image="",bg="grey").grid(row=6,column=0,padx=45)
rb2 = Label(window,image="",bg="grey").grid(row=7,column=0,padx=45)

f = open ("tmp.svg","w")
f.write(chess.svg.board(board,size=800))
f.close	
get_svg("tmp.svg")
img = Image.open("tmp.png")
cur_board_img = ImageTk.PhotoImage(img)

panel = Label(window,image=cur_board_img)
panel.bind("<Button-1>",select_piece)
panel.bind("<Button-3>",move_piece)
panel.grid(row=0,column=2,rowspan=8)

pw1 = Label(window,image="",bg="grey").grid(row=0,column=3,padx=45)
pw2 = Label(window,image="",bg="grey").grid(row=1,column=3,padx=45)
pw3 = Label(window,image="",bg="grey").grid(row=2,column=3,padx=45)
pw4 = Label(window,image="",bg="grey").grid(row=3,column=3,padx=45)
pw5 = Label(window,image="",bg="grey").grid(row=4,column=3,padx=45)
pw6 = Label(window,image="",bg="grey").grid(row=5,column=3,padx=45)
pw7 = Label(window,image="",bg="grey").grid(row=6,column=3,padx=45)
pw8 = Label(window,image="",bg="grey").grid(row=7,column=3,padx=45)
rw1 = Label(window,image="",bg="grey").grid(row=0,column=4,padx=45)
nw1 = Label(window,image="",bg="grey").grid(row=1,column=4,padx=45)
bw1 = Label(window,image="",bg="grey").grid(row=2,column=4,padx=45)
qw  = Label(window,image="",bg="grey").grid(row=3,column=4,padx=45)
kw  = Label(window,image="",bg="grey").grid(row=4,column=4,padx=45)
bw2 = Label(window,image="",bg="grey").grid(row=5,column=4,padx=45)
nw2 = Label(window,image="",bg="grey").grid(row=6,column=4,padx=45)
rw2 = Label(window,image="",bg="grey").grid(row=7,column=4,padx=45)

die = Label(window,image="",font=("Arial",45))
die.grid(row=8,column=2)

In [14]:
window.mainloop()

cur_piece =  p
move_to =  P

Die result =  6 

P captured

Pb 

R  N  B  Q  K  B  N  R  
P  p  P  P  P  P  P  P  
.  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  
.  p  p  p  p  p  p  p  
r  n  b  q  k  b  n  r  

----------------------------------------

cur_piece =  p
move_to =  B

Die result =  1 

R  N  B  Q  K  B  N  R  
P  p  P  P  P  P  P  P  
.  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  
.  p  p  p  p  p  p  p  
r  n  b  q  k  b  n  r  

----------------------------------------

cur_piece =  p
move_to =  B

Die result =  3 

R  N  B  Q  K  B  N  R  
P  p  P  P  P  P  P  P  
.  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  
.  p  p  p  p  p  p  p  
r  n  b  q  k  b  n  r  

----------------------------------------

cur_piece =  P
move_to =  .

R  N  B  Q  K  B  N  R  
.  p  P  P  P  P  P  P  
P  .  .  .  .  .  .  .  
.  .  .  